In [130]:

import random
class Sentiment:
    POSITIVE = 'POSITIVE'
    NEGATIVE = 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'



class Review:
    def __init__(self,text,rating):
        self.text=text
        self.rating=rating
        self.sentiment=self.get_sentiment()


    def get_sentiment(self):
        if self.rating <=2:
            return Sentiment.NEGATIVE
        elif self.rating == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
        
    
class review_container:
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        

    def evenly_distubute(self):
       
       

        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))

        # Ensure both lists are evenly distributed
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        # Combining negative and positive reviews to ensure an even distribution
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)  # shuffled otherwise w'll have all the postive on a side and all the negative on the other
                                       #we dont want that. we jsut need all together.(shaffled)
        

In [131]:
import json

file_name = 'Books_small_10000.json'

# Update the get_sentiment method in the Review class

reviews=[]
with open(file_name) as f:
    for line in f:
       review = json.loads(line)
       reviews.append(Review(review['reviewText'], review['overall']))
    
reviews[10].sentiment


'POSITIVE'

prep Data

In [132]:
from sklearn.model_selection import train_test_split
training,test = train_test_split(reviews,test_size=0.33,random_state=42)  # 33% of the reviews will be for testing

train_cont =  review_container(training)
test_cont =  review_container(test)



In [133]:
# Extracting the text and sentiment of each review from the training and test datasets

train_cont.evenly_distubute()
train_x = train_cont.get_text()  # Ensure train_cont is an instance of review_container
train_y = train_cont.get_sentiment()

test_cont.evenly_distubute()
test_x = test_cont.get_text()  # Ensure test_cont is an instance of review_container
test_y = test_cont.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))



436
436


bag of factroziation

In [134]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# Extracting the text of each review from the test dataset


vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)
print(train_x_vectors[0].toarray())

[[0. 0. 0. ... 0. 0. 0.]]


Classifiaction

In [135]:
#linear svm

from sklearn import svm

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

# Predicting the sentiment of the first test review
prediction = clf_svm.predict(test_x_vectors[0])

# Displaying the sentiment as POSITIVE, NEGATIVE, or NEUTRAL
print(prediction[0])  # This should display the sentiment label (e.g., POSITIVE, NEGATIVE, NEUTRAL)


POSITIVE


decision tree

In [136]:
from sklearn.tree import DecisionTreeClassifier
clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)
clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

naive bayes

In [137]:
from sklearn.naive_bayes import GaussianNB
clf_gnb =GaussianNB()
clf_gnb.fit(train_x_vectors.toarray(), train_y)
clf_gnb.predict(test_x_vectors[3].toarray())

array(['POSITIVE'], dtype='<U8')

logistic regression

In [138]:
from sklearn.linear_model import LogisticRegression
clf_log=LogisticRegression()
clf_log.fit(train_x_vectors.toarray(), train_y)
clf_log.predict(test_x_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

evalution

In [151]:
# Ensure test_y contains the sentiments of the test reviews
test_y = test_cont.get_sentiment()  # Ensure test_y is consistent with test_x_vectors
# mean accuracy
# Calculate and display the accuracy scores for SVM and Decision Tree classifiers,logestic regression,and nb
svm_score = clf_svm.score(test_x_vectors, test_y)
dec_score = clf_dec.score(test_x_vectors, test_y)
nav_score = clf_gnb.score(test_x_vectors.toarray(), test_y)
log_score = clf_log.score(test_x_vectors.toarray(), test_y)
print(f"SVM Accuracy: {svm_score}")
print(f"Decision Tree Accuracy: {dec_score}")
print(f"naive bayes accuracy:{nav_score}")
print(f"logestic regression accurancy::{log_score}")

SVM Accuracy: 0.8076923076923077
Decision Tree Accuracy: 0.6346153846153846
naive bayes accuracy:0.6610576923076923
logestic regression accurancy::0.8028846153846154


model in action!

In [142]:
#qualitative test
test_set = ['bad book do not buy','horrible waste of time','i thoroughly enjoyed this, 5 stars', ]
new_test = vectorizer.transform(test_set)
clf_svm.predict(new_test)

array(['NEGATIVE', 'NEGATIVE', 'POSITIVE'], dtype='<U8')

tuning the model(WITH GRID SEARCH)

In [152]:
# increasing the models accuracy even further
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear','rbf'),'C':(1,4,8,16,32)}
svc = svm.SVC()#The svc variable creates a basic SVM model. This is the model we want to improve.
clf=GridSearchCV(svc,parameters,cv=5)
clf.fit(train_x_vectors,train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [154]:
svm_score = clf_svm.score(test_x_vectors, test_y)
print(svm_score)

0.8076923076923077


Saving model

In [155]:
import pickle

with open("./models/sentiment_classifier.pkl","wb") as f:
    pickle.dump(clf,f)

load model

In [156]:
with open('./models/sentiment_classifier.pkl','rb') as f:
    loaded_clf = pickle.load(f)
    

In [158]:
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])

If you are the kind of person who loves to sit for hours trying to untangle a massive ball of messed up thread, read this book. If not, Amazon has several very good accounts of this event that are much better reads. If you can endure a book that constantly goes off on lengthy explanations of such things as how a telegram reaches London from Java, including the history of underwater insulation, read this book. You are a better man than I am. Way to much deviation from the story of the event. In two words,  FRUSTRATING READ.


array(['POSITIVE'], dtype='<U8')